### Generate ground truth results for T2B prompts 

In [4]:
import basico

#### Set 1

##### Precision Medicine Model

In [5]:
# generate expected answers for the basic prompts
# precision medicine model
#mod_precision = basico.load_biomodel('BIOMD0000000537')
mod_precision = basico.load_model(r'./models/BIOMD0000000537_url.xml')
print(basico.get_model_name())
# run simulation for set 1 and get the ground truth answer
tc = basico.run_time_course(2016, 2016)
tc.iloc[-1]["CRP{serum}"]

Dwivedi2014 - Crohns IL6 Disease model - Anti-IL6R Antibody


2.2691333239674267

##### Epidemiology Model

In [6]:
# epidemiology mode
#mod_epidemiology = basico.load_biomodel('BIOMD0000000971')
mod_epidemiology = basico.load_model(r'./models/BIOMD0000000971_url.xml')
print(basico.get_model_name())

Dwivedi2014 - Crohns IL6 Disease model - Anti-IL6R Antibody


##### Theoritical Model

In [7]:
# epidemiology mode
#mod_theory = basico.load_biomodel('BIOMD0000000027')
mod_theory = basico.load_model(r'./models/Tang2020.xml')
print(basico.get_model_name())

Tang2020 - Estimation of transmission risk of COVID-19 and impact of public health interventions


### Set 2 vary simulation time, interval and species to be simulated

In [8]:
def run_precision_model(interval, time, species = species, concentration = concentration):
    basico.set_species(name = species, model = mod_precision, initial_concentration = concentration)
    # check if the species was set correctly
    #print(basico.get_species(name = species, model = mod_precision)['initial_concentration'])
    # run simulations
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_precision, update_model = False)
    return tc.iloc[-1][["CRP{serum}",  "CRP{liver}", "IL6{serum}", "STAT3{gut}"]]

run_precision_model(time = 100, interval = 2016)


NameError: name 'species' is not defined

In [ ]:
def run_epidemiology_model(interval, time):
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_epidemiology, update_model = False)
    return tc.iloc[-1][["Infected", "Susceptible", "Recovered", "Hospitalised"]]


In [ ]:
def run_theory_model(interval, time):
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_theory, update_model = False)
    return tc.iloc[-1][["Mpp", "M", "Mp"]]


In [ ]:
def run_theory_model_set_parameters(interval, time, parameter, value):
    basico.set_parameters(name = parameter, initial_value = value, model = mod_theory)
    tc = basico.run_time_course(duration = time, intervals = interval, model = mod_theory, update_model = False)
    return tc.iloc[-1][["Mpp", "M", "Mp"]]

In [ ]:
run_theory_model_set_parameters(1000, 1000, "Values[k1cat]", 0.1)

Mpp    499.080250
M        0.113662
Mp       0.806087
Name: 1000.0, dtype: float64

In [ ]:
# Run with different random conditions and store in dictionary
results = {}
# adjust initial concentrations of species
initial_concentr_CRP_serum = basico.get_species(name = "CRP{serum}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_CRP_liver = basico.get_species(name = "CRP{liver}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_IL6_serum = basico.get_species(name = "IL6{serum}", model = mod_precision)['initial_concentration'].iloc[0]
initial_concentr_STAT3_gut = basico.get_species(name = "STAT3{gut}", model = mod_precision)['initial_concentration'].iloc[0]
# time 
conditions = [
    (2016, 2016, "CRP{serum}", 0.01),
    (20, 100, "CRP{serum}", 1000),
    (2016, 4032, "CRP{serum}", 2.6),
    (1000, 1000, "IL6{serum}", initial_concentr_IL6_serum*10), 
    (2016, 1000, "CRP{liver}", initial_concentr_CRP_liver*10),
    (500, 500, "STAT3{gut}", initial_concentr_STAT3_gut*10),
    (1500, 2000, "STAT3{gut}", initial_concentr_STAT3_gut*0.10)
]

for time, interval, species, concentration in conditions:
    key = f"interval_{interval}_time_{time}_species_{species}_concentration_{concentration}"
    print (concentration)
    results[key] = run_precision_model(interval, time, species, concentration)

results

0.01
1000
2.6
435628.8965511659
1583.2584678161063
6.106360135082123e-08
6.106360135082122e-10


{'interval_2016_time_2016_species_CRP{serum}_concentration_0.01': CRP{serum}    10.301993
 CRP{liver}     6.707914
 IL6{serum}     0.000641
 STAT3{gut}     9.124980
 Name: 2016.0, dtype: float64,
 'interval_100_time_20_species_CRP{serum}_concentration_1000': CRP{serum}    243.477247
 CRP{liver}    162.273954
 IL6{serum}      0.003962
 STAT3{gut}      0.291445
 Name: 20.0, dtype: float64,
 'interval_4032_time_2016_species_CRP{serum}_concentration_2.6': CRP{serum}    10.302718
 CRP{liver}     6.708469
 IL6{serum}     0.000641
 STAT3{gut}     9.124957
 Name: 2016.0, dtype: float64,
 'interval_1000_time_1000_species_IL6{serum}_concentration_435628.8965511659': CRP{serum}    91.299723
 CRP{liver}    60.894795
 IL6{serum}     0.000642
 STAT3{gut}     6.813039
 Name: 1000.0, dtype: float64,
 'interval_1000_time_2016_species_CRP{liver}_concentration_1583.2584678161063': CRP{serum}    10.328675
 CRP{liver}     6.728273
 IL6{serum}     0.000641
 STAT3{gut}     9.124149
 Name: 2016.0, dtype: floa

In [ ]:
# Run with different random conditions and store in dictionary
results = {}
# time 
conditions = [
    (50, 100),
    (100, 200),
    (180, 400),
    (500, 1000), 
    (20, 400),
    (10, 10)
]

for time, interval in conditions:
    key = f"interval_{interval}_time_{time}"
    results[key] = run_epidemiology_model(interval, time)

results

{'interval_100_time_50': Infected        1.043385e+05
 Susceptible     1.017891e+06
 Recovered       2.231583e+06
 Hospitalised    1.325140e+05
 Name: 50.0, dtype: float64,
 'interval_200_time_100': Infected        7.143353e+04
 Susceptible     1.055609e+06
 Recovered       4.586298e+06
 Hospitalised    8.688262e+04
 Name: 100.0, dtype: float64,
 'interval_400_time_180': Infected        4.031662e+04
 Susceptible     1.055585e+06
 Recovered       6.955857e+06
 Hospitalised    4.901975e+04
 Name: 180.0, dtype: float64,
 'interval_1000_time_500': Infected        4.090069e+03
 Susceptible     1.055585e+06
 Recovered       9.714202e+06
 Hospitalised    4.972991e+03
 Name: 500.0, dtype: float64,
 'interval_400_time_20': Infected        1.206681e+05
 Susceptible     3.167693e+06
 Recovered       1.634611e+05
 Hospitalised    4.076004e+04
 Name: 20.0, dtype: float64,
 'interval_10_time_10': Infected        3.413654e+03
 Susceptible     1.079620e+07
 Recovered       2.980390e+03
 Hospitalised  

In [ ]:
# Run with different random conditions and store in dictionary
results = {}
# time 
conditions = [
    (1000, 100),
    (1000, 1000),
    (180, 400),
    (500, 1000), 
    (20, 400),
    (10, 10)
]

for time, interval in conditions:
    key = f"interval_{interval}_time_{time}"
    results[key] = run_theory_model(interval, time)

results

{'interval_100_time_1000': Mpp    493.786732
 M        0.782237
 Mp       5.431030
 Name: 1000.0, dtype: float64,
 'interval_1000_time_1000': Mpp    493.786732
 M        0.782237
 Mp       5.431030
 Name: 1000.0, dtype: float64,
 'interval_400_time_180': Mpp    491.137882
 M        3.033640
 Mp       5.828478
 Name: 180.0, dtype: float64,
 'interval_1000_time_500': Mpp    493.786732
 M        0.782237
 Mp       5.431030
 Name: 500.0, dtype: float64,
 'interval_400_time_20': Mpp     45.234214
 M      424.991709
 Mp      29.774077
 Name: 20.0, dtype: float64,
 'interval_10_time_10': Mpp     17.029310
 M      460.393300
 Mp      22.577389
 Name: 10.0, dtype: float64}

In [ ]:
# run theory model with different parameters
results = {}
# time 
interval = 1000
conditions = [
    (100, 'k1cat', 0.1),
    (1000, 'k1cat', 1),
    (180, 'k1cat', 100),
    (500, 'k2cat', 10), 
    (20, 'k2cat', 0.1),
    (10, 'k2cat', 100)
]

for time, parameter, value in conditions:
    key = f"time_{time}_parameter_{parameter}_value_{value}"
    results[key] = run_theory_model_set_parameters(interval, time, parameter, value)

results

{'time_100_parameter_k1cat_value_0.1': Mpp    317.356267
 M      154.982325
 Mp      27.661408
 Name: 100.0, dtype: float64,
 'time_1000_parameter_k1cat_value_1': Mpp    494.565011
 M        0.076041
 Mp       5.358948
 Name: 1000.0, dtype: float64,
 'time_180_parameter_k1cat_value_100': Mpp    494.647985
 M        0.000758
 Mp       5.351257
 Name: 180.0, dtype: float64,
 'time_500_parameter_k2cat_value_10': Mpp    491.982136
 M        0.001140
 Mp       8.016723
 Name: 500.0, dtype: float64,
 'time_20_parameter_k2cat_value_0.1': Mpp     42.542954
 M        0.103107
 Mp     457.353939
 Name: 20.0, dtype: float64,
 'time_10_parameter_k2cat_value_100': Mpp    499.195565
 M        0.000113
 Mp       0.804322
 Name: 10.0, dtype: float64}